## External Data

- Spending now or invest for the future
- price, inflation
- income, employment, labour supply
- comsuption level, GDP
- Composite leading indicator
- adjust interest rate
- Portugal bond return and rating
- German bond return
- Stock market return and volatity
- Gold price

In [2]:
from bank_marketing import *

In [3]:
bank_mkt = import_dataset("../data/BankMarketing.csv")

## Baseline

In [11]:
drop_features = ['age',
                 'job',
                 'marital',
                 'education',
                 'default',
                 'housing',
                 'loan',
                 "duration",
                 "y"]

baseline_encoder = FunctionTransformer(cat_encode, kw_args={"drop": drop_features})
cat_clf = CatBoostClassifier(eval_metric="AUC", class_weights=[1,8], verbose=False)
benchmark(bank_mkt, baseline_encoder, cat_clf)

,Train,Validate,Test
TNR,0.859128,0.851180,0.855501
TPR,0.654766,0.645553,0.646552
bACC,0.756947,0.748366,0.751026
ROC,0.828917,0.806206,0.806289
REC,0.654766,0.645553,0.646552
PRE,0.371134,0.355078,0.362319
AP,0.516628,0.475829,0.455513


## Year

In [81]:
drop_features = ["duration",
                 "prt.bond",
                 "year",
                 "y"]

econ_encoder = FunctionTransformer(cat_encode, kw_args= {"drop": drop_features,"external":["econ"]})

In [82]:
econ_encoder.fit_transform(bank_mkt)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,deu.bond,com.lead.ind,buss.conf.idx,curr.quar.gdp,lon.gold.pri
0,56,3,1,1,0,0,0,1,5,1,1,-1,0,-1,1.1,93.994,-36.4,4.857,5191.0,4.201905,102.8580,101.55100,0.012510,571.523900
1,57,7,1,4,-1,0,0,1,5,1,1,-1,0,-1,1.1,93.994,-36.4,4.857,5191.0,4.201905,102.8580,101.55100,0.012510,571.523900
2,37,7,1,4,0,1,0,1,5,1,1,-1,0,-1,1.1,93.994,-36.4,4.857,5191.0,4.201905,102.8580,101.55100,0.012510,571.523900
3,40,0,1,2,0,0,0,1,5,1,1,-1,0,-1,1.1,93.994,-36.4,4.857,5191.0,4.201905,102.8580,101.55100,0.012510,571.523900
4,56,7,1,4,0,0,1,1,5,1,1,-1,0,-1,1.1,93.994,-36.4,4.857,5191.0,4.201905,102.8580,101.55100,0.012510,571.523900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41171,73,5,1,5,0,1,0,0,11,5,1,-1,0,-1,-1.1,94.767,-50.8,1.028,4963.6,2.533182,103.1872,99.36815,0.556357,1003.197636
41172,46,1,1,5,0,0,0,0,11,5,1,-1,0,-1,-1.1,94.767,-50.8,1.028,4963.6,2.533182,103.1872,99.36815,0.556357,1003.197636
41173,56,5,1,6,0,1,0,0,11,5,2,-1,0,-1,-1.1,94.767,-50.8,1.028,4963.6,2.533182,103.1872,99.36815,0.556357,1003.197636
41174,44,9,1,5,0,0,0,0,11,5,1,-1,0,-1,-1.1,94.767,-50.8,1.028,4963.6,2.533182,103.1872,99.36815,0.556357,1003.197636


In [83]:
cat_clf = CatBoostClassifier(eval_metric="AUC", class_weights=[1,8], verbose=False)
benchmark(bank_mkt, econ_encoder, cat_clf)

,Train,Validate,Test
TNR,0.866313,0.868799,0.863711
TPR,0.657460,0.649596,0.635776
bACC,0.761887,0.759197,0.749743
ROC,0.846035,0.813205,0.801305
REC,0.657460,0.649596,0.635776
PRE,0.384403,0.385909,0.372005
AP,0.535033,0.484452,0.450558


## Economic Indicators

In [8]:
from sklearn.impute import SimpleImputer

In [33]:
bank_1 = bank_mkt.loc[bank_mkt.index < 27682]
bank_2 = bank_mkt.loc[27682<=bank_mkt.index]
# X.loc[(27682<=X.index) & (X.index<39118), "year"] = 2009
# X.loc[39118<=X.index, "year"] = 2010
# X["year"] = X["year"].astype("int")
# X["date"] = pd.to_datetime(X[["month", "year"]].assign(day=1))
# econ = pd.read_csv("../data/Econ.csv")
# econ["date"] = pd.to_datetime(econ["date"])
# X = X.merge(econ, how="left", on="date")

In [34]:
freq_features = ["job", "marital", "education", "default", "housing", "loan"]

freq_imputer = ColumnTransformer([
    ("freq_imputer", SimpleImputer(missing_values=-1, strategy="most_frequent"), freq_features)
], remainder="passthrough")

drop_features = ["year",
                 "date",
                 "duration",
                 "y"]

econ_encoder = FunctionTransformer(cat_encode, kw_args= {"drop": drop_features,"external":["econ"]})

econ_transformer = make_pipeline(econ_encoder, freq_imputer)

In [20]:
cat_clf = CatBoostClassifier(eval_metric="AUC", class_weights=[1,20],  verbose=False)
benchmark(bank_1, econ_encoder, cat_clf)

,Train,Validate,Test
TNR,0.747790,0.731910,0.724236
TPR,0.859977,0.406542,0.350746
bACC,0.803884,0.569226,0.537491
ROC,0.886125,0.605418,0.558458
REC,0.859977,0.406542,0.350746
PRE,0.147725,0.071487,0.060763
AP,0.369321,0.095883,0.077659


In [36]:
bank_2.y.value_counts()

False    10194
True      3300
Name: y, dtype: Int64

In [35]:
cat_clf = CatBoostClassifier(eval_metric="AUC", class_weights=[1,4],  verbose=False)
benchmark(bank_2, econ_encoder, cat_clf)

,Train,Validate,Test
TNR,0.707235,0.692826,0.690535
TPR,0.819602,0.778409,0.818182
bACC,0.763419,0.735618,0.754358
ROC,0.852595,0.803086,0.814832
REC,0.819602,0.778409,0.818182
PRE,0.475419,0.450658,0.461144
AP,0.686182,0.591620,0.580357


In [47]:
X_1 = econ_transformer.fit_transform(bank_1)

In [55]:
y_1 = bank_1.y.astype("int").to_numpy()

In [59]:
y_pred = cat_clf.predict(X_1)
y_score = cat_clf.predict_proba(X_1)[:,1]
roc_auc_score(y_1, y_score)

0.5501408757683613

In [21]:
from xgboost import XGBClassifier

In [27]:
xgb_clf = XGBClassifier(max_depth=3, gamma=1, min_child_weight=1, scale_pos_weight=20)
benchmark(bank_1, econ_encoder, xgb_clf)

,Train,Validate,Test
TNR,0.659944,0.643654,0.640919
TPR,0.753792,0.518692,0.477612
bACC,0.706868,0.581173,0.559265
ROC,0.782416,0.606104,0.575228
REC,0.753792,0.518692,0.477612
PRE,0.101270,0.068816,0.063366
AP,0.206328,0.105315,0.076801
